# Post-process ADM1 summary results

For loading in oi-risk-vis interface.

In [ ]:
import os
from collections import defaultdict
from glob import glob

import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
cyclone_fnames = glob('../results/*/Cyclone/*.csv')
fluvial_fnames = glob('../results/*/Fluvial/*/*.csv')
coastal_fnames = glob('../results/*/Coastal/*/*.csv')

In [ ]:
fname = fnames[0]

In [ ]:
cols = set()
for fname in tqdm(cyclone_fnames):
    df = pd.read_csv(fname, nrows=1)
    cols = cols | set(df.columns)

## Cyclone

In [ ]:
dfs = []
for fname in tqdm(cyclone_fnames):    
    sector = os.path.basename(os.path.dirname(os.path.dirname(fname)))
    df = pd.read_csv(fname)
    df['sector'] = sector
    dfs.append(df)
cyclone = pd.concat(dfs).set_index(['sector','region_name'])

In [ ]:
def split_df_by_rp_columns(df):
    df_rp = df[[c for c in df.columns if 'RP' in c]]
    df_exp = df[[c for c in df.columns if 'RP' not in c]]
    return df_rp, df_exp

In [ ]:
cyclone_rp, cyclone_exp = split_df_by_rp_columns(cyclone)

In [ ]:
def tidy_df(df, id_vars, new_id_vars):
    # melt to get a long dataframe with "variable" and "value"
    # columns, where "variable" is taken from each input column name
    df_tidy = df \
        .reset_index() \
        .melt(id_vars=id_vars)
    
    # create a column from each new_id_var 
    # by splitting the "variable" names on "_"
    for i, varname in enumerate(new_id_vars):
        df_tidy[varname] = df_tidy.variable.apply(lambda d: d.split("_")[i+1])
        
    # keep only the first element as the actual variable name 
    df_tidy['variable'] = df_tidy.variable.apply(lambda d: d.split("_")[0])
    
    # parse an integer from return period, if present
    if 'rp' in new_id_vars:
        df_tidy.rp = df_tidy.rp.apply(lambda d: int(d.replace("RP", "")))
        
    # pivot back out to columns for each variable
    df_tidy = df_tidy.pivot_table(index=(id_vars + new_id_vars), columns='variable')
    df_tidy.columns = [v for _, v in df_tidy.columns]
    return df_tidy

In [ ]:
cyclone_rp_tidy = tidy_df(
    cyclone_rp, id_vars=['region_name', 'sector'], new_id_vars=['hazard', 'rp'])
cyclone_rp_tidy.head()

In [ ]:
cyclone_exp_tidy = tidy_df(cyclone_exp, id_vars=['region_name', 'sector'], new_id_vars=['hazard'])

In [ ]:
cyclone_exp_tidy.head()

In [ ]:
cyclone_rp_tidy.to_csv('../results/cyclone_rp.csv')

In [ ]:
cyclone_exp_tidy.to_csv('../results/cyclone_exp.csv')

## Flooding

In [ ]:
dfs = defaultdict(list)
for fname in tqdm(fluvial_fnames):
    climate = os.path.basename(os.path.dirname(fname))
    sector = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(fname))))
    df = pd.read_csv(fname)
    df['sector'] = sector
    df['rcp'] = climate
    dfs[climate].append(df)
fluvial_hist = pd.concat(dfs['historical']).set_index(['region_name','sector','rcp'])
fluvial_fut  = pd.concat(dfs['rcp4p5'] + dfs['rcp8p5']).set_index(['region_name','sector','rcp'])

In [ ]:
fluvial_fut_rp, fluvial_fut_exp = split_df_by_rp_columns(fluvial_fut)

In [ ]:
fluvial_fut_exp_tidy = tidy_df(
    fluvial_fut_exp, 
    id_vars=['region_name', 'sector', 'rcp'],
    new_id_vars=['gcm', 'epoch'])

In [ ]:
fluvial_fut_exp_tidy.head()

In [ ]:
fluvial_fut_rp_tidy = tidy_df(
    fluvial_fut_rp, 
    id_vars=['region_name', 'sector', 'rcp'],
    new_id_vars=['gcm', 'epoch', 'rp'])
fluvial_fut_rp_tidy.head()

In [ ]:
fluvial_hist_rp, fluvial_hist_exp = split_df_by_rp_columns(fluvial_hist)

In [ ]:
fluvial_hist_exp_tidy = tidy_df(
    fluvial_hist_exp, 
    id_vars=['region_name', 'sector', 'rcp'],
    new_id_vars=['gcm', 'epoch'])

fluvial_hist_exp_tidy.head()

In [ ]:
fluvial_hist_rp_tidy = tidy_df(
    fluvial_hist_rp, 
    id_vars=['region_name', 'sector', 'rcp'],
    new_id_vars=['gcm', 'epoch', 'rp'])
fluvial_hist_rp_tidy.head()